In [119]:

# !pip install tweepy
#!pip install kafka-python

#!pip install git+https://github.com/dpkp/kafka-python.git

In [1]:
import tweepy
from tweepy import StreamingClient,OAuth2BearerHandler

from kafka import KafkaProducer
import json

In [2]:

producer = KafkaProducer(bootstrap_servers='localhost:9092') #Same port as your Kafka server
topic_name = "tweet-data"

In [123]:

consumer_key = "excGmpgYvQTjLeMgXWHxFIczo"
consumer_secret = "uIj0ldQKzfi7hGU3Fb6HC4vloimoB7cMWatnF2LoFZ5MswBAMy"
access_token = "1853216357006954496-MbnigHotkFXYoKuXdBoPln9UED7BbZ"
access_token_secret = "A1UyV3fyjUxZfQ2xLXuaIeUuoUaTPx8HYlRxlulxBCUmD"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAD8WwwEAAAAAKqFi8rire0kRaFL%2FkfmAZteY%2BBY%3DsiPp46hvu8fktNbpyhYj3N0lOtflCV66FphbsB1HHEOo3AQfOa"

In [3]:

import json

class TweetsListener:
    """Process fetched tweets and send them to Kafka."""

    def __init__(self, producer, topic_name):
        self.producer = producer
        self.topic_name = topic_name

    def process_tweet(self, tweet_data):
        """Process tweet data and send it to Kafka."""
        try:
            # Extract relevant fields
            tweet_text = tweet_data.get("text", "")
            tweet_id = tweet_data.get("id", "")
            author_id = tweet_data.get("author_id", "")
            
            # Prepare output data
            out_data = {
                "tweet": tweet_text.replace("\n", ""),
                "user": author_id,
                "tweet_id": tweet_id
            }
            print("Sending to Kafka:", out_data)
            
            # Send tweet data to Kafka topic
            self.producer.send(self.topic_name, json.dumps(out_data))
        except Exception as e:
            print("Error processing tweet:", e)


# from tweepy import OAuthHandler
# from .References import References

class TwitterAuth():
    """SET UP TWITTER AUTHENTICATION"""
    def __init__(self, bearer_token):
        self.bearer_token = bearer_token

    def authenticateTwitterApp(self):
       
        return self.bearer_token


import requests
import time
# from .TwitterAuth import TwitterAuth
# from .TweetsListener import TweetsListener
# from .References import References

class TwitterStreamer():
    """Class to handle fetching recent tweets and sending to Kafka."""

    def __init__(self, producer):
        self.BEARER_TOKEN="AAAAAAAAAAAAAAAAAAAAAD8WwwEAAAAAKqFi8rire0kRaFL%2FkfmAZteY%2BBY%3DsiPp46hvu8fktNbpyhYj3N0lOtflCV66FphbsB1HHEOo3AQfOa"
        self.TOPIC_NAME='tweets_topic'
        self.TRACK_TWEET='Proj_DirectedStudies'
        self.twitterAuth = TwitterAuth(self.BEARER_TOKEN)
        self.producer = producer
        self.listener = TweetsListener(producer, self.TOPIC_NAME)
        self.last_tweet_id = None  # Store the ID of the last fetched tweet
        # Twitter recent search API endpoint
        self.search_url = "https://api.twitter.com/2/tweets/search/recent"
        
    def fetch_recent_tweets(self):
        """Fetch recent tweets based on the configured query parameters."""
        query_params = {
            "query": self.TRACK_TWEET,
            "expansions": "author_id",
            "tweet.fields": "created_at,lang",
            "user.fields": "username",
            "max_results": 10  # Number of tweets to retrieve per request
        }
        if self.last_tweet_id:
            query_params["since_id"] = self.last_tweet_id
        
        headers = {
            "Authorization": f"Bearer {self.twitterAuth.bearer_token}",
            "User-Agent": "v2TweetSearch"
        }

        response = requests.get(self.search_url, headers=headers, params=query_params)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            return []
        
        return response.json().get("data", [])

    def stream_tweets(self):
        """Fetch and process tweets in a loop, simulating a continuous stream."""
        while True:
            try:
                tweets = self.fetch_recent_tweets()
                
                for tweet_data in tweets:
                    self.listener.process_tweet(tweet_data)
                
                time.sleep(10)  # Wait for 30 seconds before fetching again

                if tweets:
                    self.last_tweet_id = tweets[0]["id"]
            except Exception as e:
                print("Error fetching tweets:", e)
                time.sleep(5)  # Wait before retrying



In [4]:
# main.py
from kafka import KafkaProducer


if __name__ == "__main__":
    # Set up Kafka producer
    kafka_broker = 'localhost:9092'  # Replace with your Kafka broker address
    producer = KafkaProducer(
        bootstrap_servers=[kafka_broker],
        value_serializer=lambda v: v.encode('utf-8')  # Serialize messages to UTF-8
    )
    
    # Create TwitterStreamer instance
    streamer = TwitterStreamer(producer)
    
    # Start streaming tweets
    streamer.stream_tweets()


Sending to Kafka: {'tweet': '@Proj_DirectedStudies this for test', 'user': '1853216357006954496', 'tweet_id': '1854617677396111511'}


KeyboardInterrupt: 